In [ ]:
install.packages("rvest")
install.packages("dplyr")
install.packages("stringr")

In [1]:

library(rvest)
library(dplyr)
library(stringr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
#loading the website:
link <- "https://www.cia.gov/the-world-factbook/field/urbanization/"
page <- read_html(link)


#extracting the information from wesite:
country <- page %>% html_nodes(".content-area-content a") %>% html_text()
country_urban_rate <- page %>% html_nodes("li p") %>% html_text()

#extracting the urbanisation rate by ountry:
urbanisation_rate <-  str_match(country_urban_rate, "urban population:\\s*(.*?)\\s*% of total population")[,2]
urbanisation_yearly_change <-  str_match(country_urban_rate, "rate of urbanization:\\s*(.*?)\\s*% annual rate of change")[,2]

#creating a dataframe:
urban_df <- data.frame(country, urbanisation_rate, urbanisation_yearly_change)


In [ ]:
#creating population dataframe:
#loading the website:
pop_link <- "https://www.cia.gov/the-world-factbook/field/population/country-comparison?fbclid=IwAR2jG4pZYN8gcvnSppY9VOROoqQIiAA48EIVy24LKpwlEpoHHTNJJcalVmU"
pop_page <- read_html(pop_link)


#extracting the information from wesite:

pop_table <- pop_page %>% html_node("table") %>% html_table()

colnames(pop_table) <- c("rank", "country", "population", "date_of_Information")


In [ ]:
#check for the difference in the two list:
setdiff(pop_table$country, urban_df$country)
setdiff(urban_df$country, pop_table$country)

In [ ]:
#merging datasets by country:

df <- merge(pop_table, urban_df, by= "country")

In [ ]:
#creating population density to match population data:
size_link <- "https://www.cia.gov/the-world-factbook/field/area/country-comparison"
size_page <- read_html(size_link)


#extracting the information from wesite:

size_table <- size_page %>% html_node("table") %>% html_table() %>% select(1:3)

colnames(size_table) <- c("rank", "country", "size")


In [ ]:

#difference:
setdiff(size_table$country, df$country)

In [ ]:
#merge the dataframe and create population density based on size and population
f <- merge(df, size_table, by= "country")
str(df)

df <- df %>% mutate(density = round(as.numeric(gsub(",","",population))/ as.numeric(gsub(",","",size)), 2))


df <- df %>% select(c(1,3,5,6, 9))

df

In [ ]:
#save DF
write.csv(df, "DataFrame.csv")